In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.db as db

import numpy as np
import sqlalchemy as sa
import pathlib

import pandas as pd

In [ ]:
sm = snakemake

## Aim
This notebook contains all the processing steps that alter the database, e.g. setting the valid_images and valid_objects as well as debarcoding



TODO:
- Filter non-attached
- Filter everything with an area < 10 pixels


In [ ]:
%matplotlib inline

In [ ]:
class C:
    fn_config = sm.input.fn_config   

In [ ]:
bro = spb.get_bro(C.fn_config)



In [ ]:
bro.data.reset_valid_images()
bro.data.reset_valid_objects()

In [ ]:
bro.session.commit()

Initial cell and image numbers

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())
print(n_img, n_obj)

Add filters

In [ ]:
%%time
outsphere = bro.filters.membership.add_issphere(minfrac=0.9, drop=True)

In [ ]:
%%time
outamb = bro.filters.membership.add_isambiguous(distother=-20, drop=True)

In [ ]:
%%time

outamb = bro.filters.membership.add_issmall(minpix=10, drop=True, object_type='cell')

In [ ]:
%%time

outamb = bro.filters.membership.add_isnotborder(borderdist=2)

Remove objects from 'valid_objects' according to these filters

In [ ]:

def calc_stats_and_apply(tup):
    q_fil = bro.filters.objectfilterlib.get_combined_filterquery([tup])

    # Calculate some additional statistics
    q_base= (bro.session.query(db.objects)
            .filter(db.objects.object_type=='cell'))
    q_tot = (q_base
            .filter(db.objects.object_id.in_(q_fil)))
    n_tot = q_tot.count()
    n_valid = q_tot.join(db.valid_objects).count()
    print(f'''{tup[0]} equal to {tup[1]} filters total {n_tot} cells
            of which {n_valid} were previously valid\n''')
    # Apply filter
    bro.filters.objectfilterlib.delete_by_filter(q_fil)



In [ ]:
for tup in [
    ('is-sphere', False),
    ('is-ambiguous', True),
    ('is-small', True),
    ('is-notborder', False)
            ]:
    calc_stats_and_apply(tup)

In [ ]:
%%time
outamb =bro.filters.membership.add_ismaincomponent(drop=True)

In [ ]:
calc_stats_and_apply(('is-maincomponent', False))

In [ ]:

q_meas = (bro.data.get_measmeta_query()
          .filter(db.ref_planes.channel_name == 'prob-hq',
                  db.measurement_names.measurement_name == 'MeanIntensity',
                 db.stacks.stack_name == 'ProbQc')
          .add_column(db.ref_stacks.scale)
         )
q_obj = (bro.data.get_objectmeta_query(valid_objects=False, valid_images=False)
        .filter(db.objects.object_type == 'cell')
        )
d=bro.io.objmeasurements.get_measurements(q_obj=q_obj, q_meas=q_meas)
d_fil = pd.DataFrame({db.object_filters.filter_value.key: d.X[:].squeeze() > 0.75,
             db.object_filters.object_id.key: d.obs.index.astype(int)})

bro.filters.objectfilterlib.write_filter_to_db(d_fil, 'is-qchq')

In [ ]:
calc_stats_and_apply(('is-qchq', False))

Look at what is left over

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())
print(n_img, n_obj)

In [ ]:
(bro.session.query(db.objects)
.filter(db.objects.object_type == 'cell').count()
)

In [ ]:

q_fil=bro.filters.objectfilterlib.get_combined_filterquery([('is-sphere', True) ])
stmt = bro.session.query(db.valid_objects).filter(db.valid_objects.object_id.in_(q_fil))
bro.doquery(stmt).shape



## Remove image ids without any valid cells

In [ ]:
from sqlalchemy import func

In [ ]:
min_cells = 25
sq_stat = (bro.session.query(db.objects.image_id, func.count(db.objects.object_id).label('count'))
     .join(db.valid_objects)
     .filter(db.objects.object_type == 'cell')
     .group_by(db.objects.image_id)
).subquery()

stmt = (bro.session.query(db.valid_images)
    .filter(~(db.valid_images.image_id.in_(bro.session.query(sq_stat.c.image_id).filter(sq_stat.c.count > min_cells).subquery())))
    )

print(f'{stmt.count()} images have less than {min_cells} single valid cells.')



In [ ]:
ax = (bro.doquery(bro.session.query(sq_stat))['count'].map(np.sqrt)).hist(bins=100)
ax.axvline(np.sqrt(min_cells), color='red')

In [ ]:
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())
print(n_img, n_obj)

In [ ]:
stmt.delete(synchronize_session='fetch')
bro.session.commit()


# This adds mean neightbourhood intensity

In [ ]:
import numpy as np

In [ ]:
%%time
bro.processing.nb_aggregation.add_nb_measurement('NbMean',np.mean, object_type='cell', measurement_name='MeanIntensityComp',
                                                    stack_name='FullStackFiltered')

In [ ]:
%%time
bro.processing.nb_aggregation.add_nb_measurement('NbMax',np.max, object_type='cell', measurement_name='MeanIntensityComp',
                                                    stack_name='FullStackFiltered')

In [ ]:
%%time
bro.processing.nb_aggregation.add_nb_measurement('NbAllMax',np.max, object_type='cell', measurement_name='MeanIntensity',
                                                    stack_name='ProbPos',valid_objects=False)

In [ ]:
%%time
bro.processing.nb_aggregation.add_nb_measurement('NbMax',np.max, object_type='cell', measurement_name='MeanIntensity',
                                                    stack_name='ProbPos',valid_objects=True)

In [ ]:
%%time
bro.processing.nb_aggregation.add_nb_measurement('NbAllMax',np.max, object_type='cell', measurement_name='MeanIntensityComp',
                                                    stack_name='FullStackFiltered',valid_objects=False)
